Имеются данные о поведении клиентов и о расторжении договоров с банком. Необходимо составить прогноз о возможном уходе клиента.
Обучим модель, рассчитывающую потенциальное поведение клиента.
Результаты предсказания модели будем проверять F1-мерой. Минимальное приемлемое ее значение - 0,59.

**1  Подготовка данных**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
data=pd.read_csv('/datasets/Churn.csv')
print(data.info())
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [3]:
data.columns = data.columns.str.lower()

In [4]:
print(data['tenure'].isna().sum()) 

909


In [5]:
data = data.dropna(subset=['tenure'])
print(data['tenure'].isna().sum()) 

0


In [6]:
data['tenure'] = data['tenure'].astype('int')

Для дальнейшего обучения моделей все категориальные столбцы необходимо преобразовать в числительные. Используем метод One-Hot Encoding. Предаврительно разделим выборку на обучающую, валидационную и тестовую выборки. Столбец с номерами, номерами ID, фамилиями можно удалить, поскольку эти значения на обучение модели не повлияют.

In [7]:
data=data.drop(['surname', 'rownumber', 'customerid'], axis=1)
data_ohe = pd.get_dummies(data, drop_first = False)
display(data_ohe.head(3))

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_France,geography_Germany,geography_Spain,gender_Female,gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0


**2  Исследование задачи**

Исследуем баланс классов.

In [9]:
print(data_ohe.exited.value_counts());

0    7237
1    1854
Name: exited, dtype: int64


Столбец с целевым признаком несбалансирован. Попробуем обучить модель на таких данных

In [10]:
target = data_ohe['exited']
features = data_ohe.drop(['exited'], axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify = data_ohe['exited'])
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify = target_valid)

Найдем результаты работы модели - DecisionTreeClassifier.

In [11]:
best_result=0
depth_1 = 0
for depth in range(1, 8):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    result_1 = f1_score(target_valid, predictions_valid)
    if result_1 > best_result:
        best_result = result_1
        depth_1 = depth
print(best_result, depth_1)

0.5862646566164155 7


Мера F1 показывает соотношение полноты и точности. Полученное значение F1 показывает, что положительный класс прогнозируется не слишком хорошо, это значение ощутимо меньше, чем необходимое.

In [12]:
model2 = LogisticRegression(random_state = 12345, solver = 'liblinear', max_iter=1000)
model2.fit(features_train, target_train)
predicted_valid = model2.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f1)

0.07637231503579953


F1 - мера показывает, что даннае модель недостаточно хорошо предсказывают возможные варианта ухода клиента. Попробуем улучшить исходные данные, чтобы улучшить результаты работы модели

**3  Борьба с дисбалансом**

Ранее уже выяснили, что положительных ответов достаточно мало. Можем сбалансировать данные, добавив положительные ответы.

In [13]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [14]:
best_result=0
depth = 0
for depth in range(1, 9):
    features_upsampled, target_upsampled = upsample(features_train, target_train, 2)
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_upsampled, target_upsampled) 
    predicted_valid = model.predict(features_valid)
    result = f1_score(target_valid, predicted_valid)
    if result > best_result:
        best_result = result
        depth_1 = depth
print(best_result, depth_1)

0.6022099447513812 5


In [15]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear', max_iter=5000)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model. predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f1)

0.36305732484076436


Попробуем балансировку данных с уменьшением отрицательных вариантов.

In [16]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [17]:
best_result=0
depth = 0
for depth in range(1, 8):
    features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_downsampled, target_downsampled) 
    predicted_valid = model.predict(features_valid)
    result = f1_score(target_valid, predicted_valid)
    if result > best_result:
        best_result = result
        depth_1 = depth
print(best_result, depth_1)

0.4801223241590214 7


In [18]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear', max_iter=1000)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f1)

0.3677452491011813


Попробуем придать объектам более редкого класса больший вес - установим величину class weight = balanced.

In [19]:
best_result=0
depth = 0
for depth in range(1, 9):
    model = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=depth) 
    model.fit(features_train, target_train) 
    predicted_valid = model.predict(features_valid)
    result = f1_score(target_valid, predicted_valid)
    if result > best_result:
        best_result = result
        depth_1 = depth
print(best_result, depth_1)

0.583084577114428 6


In [20]:
model = LogisticRegression(random_state = 12345, solver = 'lbfgs', max_iter=1000, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model. predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f1)

0.4556737588652482


Наилучшие данные метрики F1 получены с использованием балансировки данных увеличением выборки.
Для тестирования модели возьмем именно этот - самый успешный вариант.

**4  Тестирование модели**

In [21]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 2)
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_upsampled, target_upsampled)
predicted_test = model. predict(features_test)
f1 = f1_score(target_test, predicted_test)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_valid)
print(f1)
print(auc_roc)

0.5921787709497207
0.8131515167309498


Обучили модель предсказания дальнейшего поведения клиента. Мера F1 = 0.5921787709497207, что соответствует требованиям заказчика. Дополнительно проверим значени метрики auc_roc. В нашем случае, значение этой метрики 0.8131515167309498, что достаточно близко к 1 (максимально возможному результаты для данной метрики).

**Заключение**

Для банка необходимо составить прогноз о возможном уходе клиента из банка в ближайшее время. На основании исторических данных о поведении клиентов и расторжении договоров с банком, построим модели, прогнозирующие требуемые данные.

1. Подготовка данных

Рассмотрим, имеющиеся данные. Имеется таблица с 10000 строк, в столбцах расположены личные данные клиентов банка: фамилии, кредитный рейтинг, страна проживания и т.д. Для начала приведем все названия столбцов к привычному написанию – все буквы строчные. В столбце tenure – данные о количестве лет сотрудничества клиента с банком – имеются пропущенные значения. Присваиваем им значение 0, поскольку вероятнее всего, пропущены значения в тех строках, где клиент является клиентом банка менее 1 года. Значения в этом столбце приведем к целочисленным данным. Значения во всех категориальных столбцах приведем к численным – присвоим категории для значений фамилий, страны проживания и пола клиента.

2. Исследование задачи

Баланс классов в столбце с целевым признаком не соблюден. Но для начала проведем обучение модели на несбалансированных данных. Данные поделили на три части: обучающую, валидационную и тестовую. Обучающая часть - самая большая выборка – составила 60% от изначально предоставленных данных. Оставшуюся часть поделили по 20% от общих данных для валидации модели и тестирования. Для сравнения используем модели обучения: DecisionTreeClassifier (дерево решений) и LogisticRegression (логистическая регрессия). Основная определяющая метрика качества предсказаний модели – F1.

3. Борьба с дисбалансом

Попробуем сбалансировать выборку и сравним результаты обучения модели на сбалансированных данных. Первый способ – увеличить количество положительных классов. Берем значения =1, увеличиваем в несколько раз и перемешиваем среди остальных данных. Результаты обучения модели на таких данных для дерева решений - 0.6022099447513812, логистической регрессии - 0.36305732484076436. Второй способ - уменьшаем количество отрицательных классов. Берем значения =0, уменьшаем их количество до 0,1, и перемешиваем среди остальных данных. Результаты обучения модели на таких данных для дерева решений - 0.4801223241590214, логистической регрессии - 0.3677452491011813. Что чуть хуже, чем при предыдущем балансировании. Третий способ – повысим важность класса, установим величину class weight = balanced. Результаты обучения модели на таких данных для дерева решений - 0.583084577114428, логистической регрессии - 0.4556737588652482. Наилучший результат был достигнут моделью Дерево решений при увеличении выборки. Для дальнейшего тестирования используем эту модель.

4. Тестирование модели

Возьмем параметры, давшие наилучший результат метрики F1, и протестируем на тестовой выборке. Дополнительно исследуем метрику auc-roc. Достигли значения метрики F1 - 0.5921787709497207, auc_roc = 0.8131515167309498. Это позволяет использовать данную модель для дальнейшего предсказания поведения клиентов.